## Dependency

In [45]:
import os
import json
import pickle
import random
import warnings
import gc

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import transformers
from einops import rearrange
from scipy.stats import spearmanr
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.neural_network import MLPRegressor
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import torch
import torch.nn.functional as F

from baukit import Trace, TraceDict
from custom_llama import llama # modified code to access attention head outputs

warnings.filterwarnings("ignore")

In [46]:
HUGGINGFACE_TOKEN = os.environ.get("HF_ACCESS_TOKEN")

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
os.environ["HTTP_PROXY"] = "http://127.0.0.1:10808"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:10808"

### Helpers

In [49]:
def entropy(probs, base=None):
    """
    Compute Shannon entropy of a probability distribution.

    Args:
        probs (np.array): 1D array of probabilities (should sum to 1)
        base (float or None): Logarithm base. If None, uses natural log.

    Returns:
        float: Entropy value
    """
    probs = np.array(probs)
    probs = probs[probs > 0]  # filter zero to avoid log(0)
    log_fn = np.log if base is None else lambda x: np.log(x) / np.log(base)
    return -np.sum(probs * log_fn(probs))

In [127]:
def load_tokenizer_and_model(model_name, device=None, hf_token=None):
    """
    Loads a tokenizer and model for causal LM generation.
    """
    tokenizer = AutoTokenizer.from_pretrained(
        model_name, cache_dir='./model', use_auth_token=hf_token
    )
    model = llama.LlamaForCausalLM.from_pretrained(
        model_name,
        cache_dir='./model',
        torch_dtype=torch.float16 if device.startswith('cuda') else torch.float32,
        low_cpu_mem_usage=True,
        use_auth_token=hf_token
    ).to(device).eval()
    return tokenizer, model


def extract_attention_head_activations(model, prompts):
    """
    Extracts prefill attention head activations for a list of prompts.
    """
    HEADS = [f"model.layers.{i}.self_attn.head_out" for i in range(model.config.num_hidden_layers)]
    head_wise_hidden_states_list = []

    for prompt in tqdm(prompts, total=len(prompts)):
        with torch.no_grad():
            with TraceDict(model, HEADS) as ret:
                output = model(prompt.to('cuda'), output_hidden_states=True, output_attentions=True)
                head_wise_hidden_states = [ret[head].output.squeeze().detach().cpu() for head in HEADS]
                head_wise_hidden_states = torch.stack(head_wise_hidden_states, dim=0).squeeze().numpy()
                head_wise_hidden_states_list.append(head_wise_hidden_states[:, :, :])

    features = [
        rearrange([states[:, -1, :]], 'b l (h d) -> b l h d', h=model.config.num_attention_heads)
        for states in head_wise_hidden_states_list
    ]
    return np.stack(features, axis=0)


def extract_and_save_features(model, tokenizer, model_name, df_prompts, output_dir="results_replication", output_suffix="anes_features", device=None):
    """
    Given a DataFrame with columns ['prompt', 'label'], extract attention features and save to pickle.
    """
    # Tokenize all prompts
    encoded = [tokenizer(p, return_tensors='pt')['input_ids'].to(device) for p in df_prompts['prompt']]
    labels = df_prompts['label'].values

    # Extract features
    features = extract_attention_head_activations(model, encoded)

    # Save
    out_path = f"{output_dir}/{model_name.replace('/', '_')}_{output_suffix}.pkl"
    with open(out_path, 'wb') as f:
        pickle.dump((features, labels), f)
    print(f"Saved features to {out_path}")


def clean_up(device=None):
    """
    Clean up the model and GPU memory.
    """
    gc.collect()
    if device:
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()


def extract_head_out_per_token(
    model, tokenizer, prompts,
    candidate_answers=None,
    max_new_tokens=32,
    output_only=False,
    device=None):
    """
    Extracts attention head outputs per token and optionally probabilities for candidate answers.

    Args:
        model: HuggingFace transformer model
        tokenizer: Corresponding tokenizer
        prompts: List of prompt strings
        candidate_answers: List of string candidates (e.g., ["1", "2", ..., "7"]) or None
        max_new_tokens: Number of tokens to generate

    Returns:
        List of dictionaries, each with:
            - 'features': np.array (T, L, H, D)
            - 'token_ids': np.array of generated token IDs
            - 'answer': decoded generated string
            - 'candidate_logits': np.array (len(candidate_answers),) if applicable
            - 'candidate_probs': np.array (len(candidate_answers),) if applicable
    """
    num_layers = model.config.num_hidden_layers
    num_heads = model.config.num_attention_heads
    hidden_size = model.config.hidden_size
    head_dim = hidden_size // num_heads

    HEADS = [f"model.layers.{i}.self_attn.head_out" for i in range(num_layers)]

    # Prepare candidate token IDs
    candidate_token_ids = None
    if candidate_answers is not None:
        candidate_token_ids = [tokenizer.convert_tokens_to_ids(ans) for ans in candidate_answers]
        candidate_token_ids = torch.tensor(candidate_token_ids, device=device)

    results = []

    for prompt in tqdm(prompts):
        input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)

        token_outputs_by_layer = [[] for _ in range(num_layers)]

        def make_hook(layer_idx):
            def hook_fn(module, inp, out):
                if output_only:
                    token_outputs_by_layer[layer_idx].append(out.squeeze(0)[-1].detach().cpu())
                else:
                    token_outputs_by_layer[layer_idx].extend(out.squeeze(0).detach().cpu())
            return hook_fn

        # Register hooks
        hooks = []
        for i, name in enumerate(HEADS):
            module = dict(model.named_modules())[name]
            hooks.append(module.register_forward_hook(make_hook(i)))

        # Generate tokens and optionally return scores
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                return_dict_in_generate=True,
                output_scores=(candidate_answers is not None),
                output_attentions=False
            )

        for h in hooks:
            h.remove()

        # Token output
        generated_ids = outputs.sequences.squeeze(0).cpu().numpy()
        decoded = tokenizer.decode(generated_ids, skip_special_tokens=True)

        # Attention head features
        layerwise_outputs = []
        for layer_outputs in token_outputs_by_layer:
            stacked = torch.stack(layer_outputs, dim=0)  # (T, hidden_size)
            reshaped = stacked.view(stacked.size(0), num_heads, head_dim)  # (T, H, D)
            layerwise_outputs.append(reshaped)

        layerwise_outputs = torch.stack(layerwise_outputs, dim=0)  # (L, T, H, D)
        layerwise_outputs = rearrange(layerwise_outputs, 'l t h d -> t l h d')  # (T, L, H, D)

        # Package result
        result = {
            "features": layerwise_outputs.numpy(),  # (T, L, H, D)
            "token_ids": generated_ids,
            "answer": decoded,
        }

        # Extract candidate logits & probs
        if candidate_token_ids is not None and len(outputs.scores) > 0:
            first_step_logits = outputs.scores[0].squeeze(0)  # (vocab_size,)
            probs = F.softmax(first_step_logits, dim=-1)  # (vocab_size,)
            candidate_logits = first_step_logits[candidate_token_ids].cpu().numpy()
            candidate_probs = probs[candidate_token_ids].cpu().numpy()
            result["candidate_logits"] = candidate_logits
            result["candidate_probs"] = candidate_probs

        results.append(result)

    return results


def get_top_indices(performance, k=8):
    # Top-k (i, j) heads
    return np.dstack(np.unravel_index(np.argsort(performance.ravel()), (32, 32)))[0][-k:, :]


def predict_per_token_scores(results, ridge_models, performance, k):
    """
    Predict per-token scores for voter samples with variable-length inputs.

    Args:
        results: List of dictionaries with keys 'features' and 'token_ids'
        models: dict mapping (i,j) -> trained Ridge model
        performance: dict mapping (i,j) -> performance score
        k: int, number of top ridge models to use

    Returns:
        scores: List[np.ndarray] where each element has shape (T,)
    """
    top_indices = get_top_indices(performance, k=k)

    for res in results:  # sample_features: (T, L, H, D)
        sample_features = res['features']  # Shape: (T, L, H, D)
        T, L, H, D = sample_features.shape
        scores = np.zeros(T)

        for t in range(T):
            for i, j in top_indices:
                model = ridge_models[i][j]
                x = sample_features[t, i, j, :]  # Shape: (D,)
                scores[t] += model.predict(x.reshape(1, -1))[0]

        scores /= len(top_indices)
        res['scores'] = scores

    return results


def visualize_token_scores(results, tokenizer, title_prefix="Prompt", vmin=-1.0, vmax=1.0):
    """
    Displays tokens with per-token scores using color-coding.

    Args:
        results (List[Dict]): List of dictionaries with keys 'token_ids' and 'features'.
        scores (np.ndarray): Array of shape (N_prompts, T) with score values per token.
        tokenizer (transformers.PreTrainedTokenizer): HuggingFace tokenizer to decode tokens.
        title_prefix (str): Optional prefix for prompt titles.
    """    
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = plt.get_cmap('RdBu_r')

    for idx, res in enumerate(results):
        ids = res['token_ids']
        score_arr = res['scores']
        tokens = tokenizer.convert_ids_to_tokens(ids)
        spans = []
        for tok, sc in zip(tokens[1:], score_arr[: len(tokens)]):  # skip BOS token
            color = mpl.colors.to_hex(cmap(norm(sc)))
            safe_tok = (
                tok.replace('&', '&amp;')
                .replace('<', '&lt;')
                .replace('>', '&gt;')
                .replace('Ġ', ' ')  # optional: replace GPT-style space token if BPE
            )
            spans.append(
                f"<span style='background-color:{color};padding:2px;margin:1px;border-radius:3px;color:#313131;'>{safe_tok}</span>"
            )
        html = (
            '<div style="line-height:1.6;font-family:monospace;">'
            + ' '.join(spans)
            + '</div>'
        )
        display(HTML(f"<h4>{title_prefix} {idx + 1}:</h4>{html}"))
        

def plot_candidate_probabilities(results, candidate_answers, title_prefix="Prompt", figsize=(8, 4)):
    """
    Visualizes the candidate probability distributions for each prompt result.

    Args:
        results: List of result dictionaries returned by extract_head_out_per_token
        candidate_answers: List of string labels corresponding to candidate tokens
        title_prefix: Prefix used in subplot titles (e.g., "Prompt 1", "Prompt 2", ...)
        figsize: Size of each subplot figure
    """
    num_plots = len(results)

    for idx, result in enumerate(results):
        if "candidate_probs" not in result:
            print(f"Skipping prompt {idx+1}: no candidate_probs found.")
            continue

        probs = result["candidate_probs"]
        decoded_answer = result["answer"]

        plt.figure(figsize=figsize)
        bars = plt.bar(candidate_answers, probs)

        plt.title(f"{title_prefix} {idx + 1}")
        plt.ylabel("Probability")
        plt.ylim(0, 1)
        plt.xticks(rotation=0)

        # Annotate bar values
        for bar, prob in zip(bars, probs):
            plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
                     f"{prob:.2f}", ha="center", va="bottom", fontsize=9)

        plt.tight_layout()
        plt.show()


def load_ridge_models(ridge_prefix):
    """
    Load ridge models and performance scores.

    Args:
        ridge_prefix (str): Prefix for the ridge model files.
        k (int): Number of top ridge models to load.

    Returns:
        tuple: Tuple containing the loaded ridge models and performance scores.
    """
    performances = pickle.load(open(f"./results_replication/{ridge_prefix}_performance.pkl", 'rb'))
    ridge_models = pickle.load(open(f"./results_replication/{ridge_prefix}_ridge.pkl", 'rb'))
    
    return ridge_models, performances


def generate_and_score_tokens(
    model_name,
    prompts,
    model=None,
    tokenizer=None,
    max_new_tokens=512,
    hf_token=None,
    k=8,
    candidate_answers=None,
    output_only=False,
    ridge_prefix=None,
    device=None
):
    """
    Generate and visualize token-level bias scores for a list of prompts.

    Args:
        model_name (str): Name of the HuggingFace model.
        prompts (List[str]): List of user prompts.
        max_new_tokens (int): Max tokens to generate during inference.
        hf_token (str): HuggingFace token if authentication is needed.
        k (int): Number of top ridge models to use.
    """
    delete_model = False
    
    if model and tokenizer:
        model.to(device)
    else:
        tokenizer, model = load_tokenizer_and_model(model_name, device=device, hf_token=hf_token)
        delete_model = True

    # Step 1: Extract features
    results = extract_head_out_per_token(
        model, tokenizer, prompts,
        max_new_tokens=max_new_tokens,
        candidate_answers=candidate_answers,
        output_only=output_only,
        device=device
    )

    # Step 2: Load ridge models and performance scores
    if ridge_prefix is None:
        ridge_prefix = f"{model_name.replace('/', '_')}_politician"
    ridge_models, performances = load_ridge_models(ridge_prefix)

    # Step 4: Predict token-level scores
    results = predict_per_token_scores(results, ridge_models, performances, k)

    if delete_model:
        del model
        del tokenizer
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        
    return results

### Plotting

In [51]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def visualize_head_performance(model_name: str, performance_path: str, top_k: int = 20):
    """
    Load and visualize attention head performance heatmap.

    Args:
        model_name (str): Name of the model, e.g. 'meta-llama/Llama-2-7b-chat-hf'
        performance_path (str): Path to the .pkl file storing the (layer, head) performance matrix.
        top_k (int): Number of top-performing heads to print.

    Returns:
        top_indices (np.ndarray): Array of shape (top_k, 2) with top (layer, head) indices.
        top_scores (np.ndarray): Corresponding performance scores for those indices.
    """
    print(f"Model: {model_name}")
    
    # Load performance data
    safe_name = model_name.replace('/', '_')
    with open(performance_path.format(model=safe_name), 'rb') as f:
        performance = pickle.load(f)

    print("Max performance:", performance.max())

    # Plot heatmap
    plt.rcParams.update({'font.size': 20})
    plt.figure(figsize=(6, 6))
    norm = mcolors.Normalize(vmin=0, vmax=0.9)
    sorted_perf = np.sort(performance[::-1, :] * (-1), axis=1) * (-1)
    plt.imshow(sorted_perf, cmap='YlGnBu', norm=norm)
    plt.grid(False)
    plt.xlabel("Head (Sorted)")
    plt.ylabel("Layer")
    plt.xticks([])
    plt.yticks(range(0, 32, 2), [i for i in range(32, 0, -2)])
    cbar = plt.colorbar(orientation='vertical', fraction=0.0459, pad=0.04)
    cbar.outline.set_visible(False)
    for side in ['top', 'right', 'left', 'bottom']:
        plt.gca().spines[side].set_visible(False)
    plt.tight_layout()
    plt.show()

    # Extract top-k performing heads
    print("Top accuracy heads:")
    top_indices = np.dstack(np.unravel_index(np.argsort(performance.ravel()), (32, 32)))[0][-top_k:][::-1]
    print(top_indices)

    top_scores = performance[top_indices[:, 0], top_indices[:, 1]]
    print("Top accuracy scores:")
    print(top_scores)

    return top_indices, top_scores


## Data

In [52]:
df = pd.read_csv('./data_replication/HS116_members.csv')
df = df.loc[pd.notnull(df.nominate_dim1)].reset_index(drop=True)
df['first_middle_name'] = df['bioname'].apply(lambda x: x.split(', ')[1])
df['first_name'] = df['first_middle_name'].apply(lambda x: x.split(' ')[0] + ' ' if ' ' in x else x + ' ')
df['middle_name'] = df['first_middle_name'].apply(lambda x: x.split(' ')[1][0] + '. ' if ' ' in x else '')
df['last_name'] = df['bioname'].apply(lambda x: x.split(', ')[0].lower().capitalize().strip())
df['fullname'] = df['first_name'] + df['middle_name'] + df['last_name']

In [53]:
adfontesmedia = pd.read_csv('./data_replication/adfontesmedia.csv')

In [ ]:
df_anes = pd.read_csv("data_replication/anes.csv")

# Prompt builder
def build_prompt(persona, query):
    return (
        f"USER: How would {persona} in the United States in 2021 answer the following question. "
        f"{query.strip()} "
        f"ASSISTANT: As {persona}, I would answer the question with a score of "
    )

prompts_anes = []
for _, row in df_anes.iterrows():
    domain, question = row['domain'], row['query']
    for persona in df['fullname'].tolist():
        prompts_anes.append(build_prompt(persona, question))

In [55]:
# Prompt builder
def build_prompt(alignment, query):
    return (
        f"USER: Generate a statement from the {alignment} on the following question in the United States. "
        # f"{query.strip()}\n"
        # f"Do you favor or oppose transgender bathrooms?\n"
        # f"Do you favor or oppose US military aid to Ukraine?\n"
        f"Do you favor or oppose buying green socks for Christmas?\n"
        f"ASSISTANT: In 2021, the {alignment} believes that"
    )
    
prompts_socks = []
for alignment in ["liberal", "conservative"]:
    prompts_socks.append(build_prompt(alignment, None))

In [131]:
def build_prompt(query):
    return (
        f"USER: Is this statement neutral or biased to the left or right in the 2021 US political context? "
        f"\"{query.strip()}\"\n"
        f"ASSISTANT: The statement is"
    )

df_bias = pd.read_parquet("data_replication/political_ideologies_test.parquet")

prompts_bias = []
for _, row in df_bias.sample(100, random_state=42).iterrows():
    text = row['statement']
    prompts_bias.append(build_prompt(text))


## Extract features per token

In [ ]:
tokenizer, model = load_tokenizer_and_model(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device=device,
    hf_token=HUGGINGFACE_TOKEN
)

Plink failed to import tkinter.


2025-05-18 23:11:33.637107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747581093.658379  531538 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747581093.664883  531538 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 23:11:33.687395: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[2025-05-18 23:11:35,582] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


No ROCm runtime is found, using ROCM_HOME='/opt/rocm'
LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompts = [
    "USER: Write an essay about democrats and republican policies on military aid to Ukraine in the 2022 US.\nASSISTANT:",
    "USER: Repeat the following words: \"Democrats have generally been more supportive of providing military aid to Ukraine, viewing it as a crucial component of maintaining stability in Europe and countering Russian aggression. In 2022, the Democratic-led Congress passed the National Defense Authorization Act (NDAA), which included a significant increase in military aid to Ukraine. The NDAA provided $300 million in military assistance to Ukraine, including funding for weapons, training, and logistical support.\n\nIn contrast, Republicans have been more cautious in their approach to military aid to Ukraine. While some Republicans have expressed support for providing aid to Ukraine, others have been critical of the idea, arguing that it could escalate the conflict and lead to a wider war with Russia.\"\nASSISTANT:",
    "USER: Analyze the following news excerpt for political bias. Progressive leaders have once again stepped up to protect the environment, pushing back against corporate greed and right-wing obstructionism.\nASSISTANT: The excerpt is leaning to the"
]

generate_and_score_tokens(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    model=model,
    tokenizer=tokenizer,
    prompts=prompts,
    max_new_tokens=512,
    hf_token=HUGGINGFACE_TOKEN,
    k=8,
    device=device,
)

100%|██████████| 3/3 [00:27<00:00,  9.21s/it]


## ANES Inconsistency

In [ ]:
results = generate_and_score_tokens(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    model=model,
    tokenizer=tokenizer,
    prompts=prompts_anes,
    max_new_tokens=1,
    hf_token=HUGGINGFACE_TOKEN,
    k=8,
    candidate_answers=[str(i) for i in range(1, 8)],
    output_only=True,
    device=device
)

  0%|          | 4/3312 [00:00<03:37, 15.18it/s]

100%|██████████| 3312/3312 [03:30<00:00, 15.72it/s]


In [30]:
features = []
labels = []

for res in results:
    features.append(res['features'])
    labels.append(
        entropy(res['candidate_probs']) / np.log(len(res['candidate_probs']))
    )

features = np.stack(features, axis=0)
features.shape

(3312, 1, 32, 32, 128)

In [ ]:
performance = np.zeros((model.config.num_hidden_layers, model.config.num_attention_heads))
ridge_dict = {}
for i in tqdm(range(model.config.num_hidden_layers)):
    ridge_dict[i] = {}
    for j in range(model.config.num_attention_heads):
        kf = KFold(n_splits=2, shuffle=True, random_state=42)
        for train_indices, test_indices in kf.split(range(features.shape[0])):
            X_train = features[train_indices, 0, i, j, :]
            X_test = features[test_indices, 0, i, j, :]
            y_train = np.array(labels)[train_indices]
            y_test = np.array(labels)[test_indices]
            ridge_model = Ridge(alpha=1, fit_intercept=False)
            ridge_model.fit(X_train, y_train)
            ridge_dict[i][j] = ridge_model
            y_pred = ridge_model.predict(X_test)
            performance[i, j] += spearmanr(y_test, y_pred).statistic

performance /= 2
pickle.dump(performance, open("./results_replication/inconsistency_performance.pkl", 'wb'))
pickle.dump(ridge_dict, open("./results_replication/inconsistency_ridge.pkl", 'wb'))


In [ ]:
top_indices, top_scores = visualize_head_performance(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    performance_path="./results_replication/inconsistency_performance.pkl",
    top_k=20
)

In [ ]:
# plot_candidate_probabilities(
#     results,
#     candidate_answers=[str(i) for i in range(1, 8)],
#     title_prefix="Prompt",
#     figsize=(8, 4)
# )

## Politicians

1. Could there be one head for each policy domain?
2. Why is there only one head (k=1) that generates the best divergence between the liberal/conservative perspectives?
3. Why is the results averaged when adding heads? Is it because of the policy domains?
4. When I ask about what socks liberal/conservatives would buy for Christmas, I get the same results with the DW NOMINATE probes. So the probes are only sensitive to the liberal/conservative divergence, but not the actual questions or domains asked? Which kind of makes sense, since the probes are not trained on any specific domain, but rather on the general democrat/republican divergence found in the DW NOMINATE data.
5. How do I quantify the contribution of each policy domain to the liberal/conservative divergence? Is there a way to combine the policy probes to get the results of the DW NOMINATE probes?
6. Maybe I can retrain the probes without any data?
7. What about using MSE/KL Divergence for classification when the dimension is discrete? Instead of using regression when target is continuous?
8. If I ever wanted to explain the difference in task scenario, I would have to probe for utilization of knowledge directly? But that cannot be observed so cannot be probed? Maybe ask the model to generate reasoning process, fit the probe on the reasoning (comparison of sentences), and then probe for that comparison?
9. Train another probe to explain the variance?
10. How to probe for liberal/conservative divergence in the model in different layers? For example, in early layers, it understands the leftist perspective properly, but in later layers, it gets scrambled by RLHF? Can regression models be used to probe for this?
11. Probe the reasoning process for bias detection, and then probe the non-reasoning bias detection inference, see if there is any similarity?

### Prompting

In [8]:
## Generate politician simulation prompts

for model_name in ['meta-llama/Llama-2-7b-chat-hf', 'mistralai/Mistral-7B-Instruct-v0.1', 'lmsys/vicuna-7b-v1.5']:
    tokenizer = llama.LlamaTokenizer.from_pretrained(model_name, cache_dir='./model', token=HUGGINGFACE_TOKEN)
    
    # Generate prompts for politician simulation
    statements = []
    for index, row in df.iterrows():
        statements.append(tokenizer(f'USER: Generate a statement by {row["fullname"]}, a politician in the United States.\nASSISTANT: In 2019, {row["fullname"]} said that', return_tensors="pt")['input_ids'])
    pickle.dump(statements, open(f'./results_replication/{model_name.replace("/", "_")}_politicians.pkl', 'wb'))
    
    # Generate prompts for news media simulation
    statements_news = []
    for index, row in adfontesmedia.iterrows():
        statements_news.append(tokenizer(f'USER: Generate a statement from a news source in the United States.\nASSISTANT: {row["moniker_name"]} reported that', return_tensors="pt")['input_ids'])
    pickle.dump(statements_news, open(f'./results_replication/{model_name.replace("/", "_")}_news.pkl', 'wb'))

### Extracting Activations

In [ ]:
# for model_name in ['meta-llama/Llama-2-7b-chat-hf', 'mistralai/Mistral-7B-Instruct-v0.1', 'lmsys/vicuna-7b-v1.5']: #
for model_name in ['meta-llama/Llama-2-7b-chat-hf']:
    model = llama.LlamaForCausalLM.from_pretrained(model_name, cache_dir='./model', low_cpu_mem_usage=True, torch_dtype=torch.float16, token=HUGGINGFACE_TOKEN).to('cuda:0')
    # Extract activations for politicians
    statements = pickle.load(open(f'./results_replication/{model_name.replace("/", "_")}_politicians.pkl', 'rb'))
    labels = np.array(df['nominate_dim1'].astype(float))
    features = extract_attention_head_activations(model, statements)
    pickle.dump((features, labels), open(f"./results_replication/{model_name.replace('/','_')}_politician_features.pkl", 'wb'))

    # Extract activations for news media
    statements_news = pickle.load(open(f'./results_replication/{model_name.replace("/", "_")}_news.pkl', 'rb'))
    labels = np.array(adfontesmedia['bias_mean'].astype(float))
    features = extract_attention_head_activations(model, statements_news)
    pickle.dump((features, labels), open(f"./results_replication/{model_name.replace('/','_')}_news_features.pkl", 'wb'))

    del model
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

### Probing

In [ ]:
for model_name in ['meta-llama/Llama-2-7b-chat-hf']:
    features, labels = pickle.load(open(f"./results_replication/{model_name.replace('/','_')}_politician_features.pkl", 'rb'))
    tokenizer, model = load_tokenizer_and_model(model_name, device=device, hf_token=HUGGINGFACE_TOKEN)

    performance = np.zeros((model.config.num_hidden_layers, model.config.num_attention_heads))
    ridge_dict = {}
    for i in tqdm(range(model.config.num_hidden_layers)):
        ridge_dict[i] = {}
        for j in range(model.config.num_attention_heads):
            kf = KFold(n_splits=2, shuffle=True, random_state=42)
            for train_indices, test_indices in kf.split(range(features.shape[0])):
                X_train = features[train_indices, 0, i, j, :]
                X_test = features[test_indices, 0, i, j, :]
                y_train = np.array(labels)[train_indices]
                y_test = np.array(labels)[test_indices]
                ridge_model = Ridge(alpha=1, fit_intercept=False)
                ridge_model.fit(X_train, y_train)
                ridge_dict[i][j] = ridge_model
                y_pred = ridge_model.predict(X_test)
                performance[i, j] += spearmanr(y_test, y_pred).statistic

    performance /= 2
    pickle.dump(performance, open(f"./results_replication/{model_name.replace('/','_')}_politician_performance.pkl", 'wb'))
    pickle.dump(ridge_dict, open(f"./results_replication/{model_name.replace('/','_')}_politician_ridge.pkl", 'wb'))

    del model
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

# Intervention

In [41]:
import pickle
import numpy as np
from tqdm import tqdm
from einops import rearrange
import torch
from baukit import TraceDict

def generate_with_head_intervention(
    model, tokenizer,
    model_name: str,
    prompt: str,
    k: int,
    alpha: float,
    max_new_tokens: int = 200,
    results_dir: str = "./results_replication"
) -> str:
    """
    Generates model output with top-k head modulation using pre-trained Ridge coefficients.

    Args:
        model: HuggingFace transformer model
        tokenizer: Corresponding tokenizer
        model_name (str): e.g., 'meta-llama/Llama-2-7b-chat-hf'
        prompt (str): Prompt to use for generation
        topic (str): Topic string (for intermediate string formatting)
        k (int): Number of top heads to modulate
        alpha (float): Scaling coefficient for intervention
        max_new_tokens (int): Max length of generated tokens
        results_dir (str): Directory containing precomputed .pkl results

    Returns:
        str: Generated output string
    """

    # Load data
    safe_model_name = model_name.replace('/', '_')
    perf_path = f"{results_dir}/{safe_model_name}_politician_performance.pkl"
    feat_path = f"{results_dir}/{safe_model_name}_politician_features.pkl"
    ridge_path = f"{results_dir}/{safe_model_name}_ridge.pkl"

    performance = pickle.load(open(perf_path, 'rb'))
    features, labels = pickle.load(open(feat_path, 'rb'))
    trained_ridge_dict = pickle.load(open(ridge_path, 'rb'))

    # Get top-k performing heads
    top_indices = np.dstack(np.unravel_index(np.argsort(performance.ravel()), (32, 32)))[0][-k:][::-1]

    # Build focal_ridge_dict and head_dict
    focal_ridge_dict = {
        tuple(i): trained_ridge_dict[i[0]][i[1]].coef_ for i in top_indices
    }
    head_dict = {}
    for i in top_indices:
        head_dict.setdefault(i[0], []).append(i[1])

    # Define intervention hook
    def lt_modulated_vector_add(head_output, layer_name):
        layer_index = int(layer_name[len('model.layers.') : layer_name.index('.', len('model.layers.'))])
        head_output = rearrange(
            head_output.detach().cpu(), 'b s (h d) -> b s h d',
            h=model.config.num_attention_heads
        )
        for head_index in head_dict[layer_index]:
            std = np.std(features[:, 0, layer_index, head_index, :], axis=0)
            head_output[:, -1, head_index, :] += alpha * focal_ridge_dict[(layer_index, head_index)] * std
        return rearrange(head_output, 'b s h d -> b s (h d)').to('cuda')

    # Compose input and intervene
    traced_layers = [f'model.layers.{i}.self_attn.head_out' for i in sorted(set(top_indices[:, 0]))]
    with TraceDict(model, traced_layers, edit_output=lt_modulated_vector_add):
        input_ids = tokenizer(prompt, return_tensors="pt")['input_ids']
        input_ids = input_ids.to('cuda')[0][:-1].unsqueeze(0)  # remove last token to match generation format

        model_gen_tokens = model.generate(input_ids, max_length=input_ids.shape[1] + max_new_tokens)
        model_gen_str = tokenizer.decode(model_gen_tokens[0], skip_special_tokens=True).strip()

    return model_gen_str

In [125]:
import pickle
from typing import Union, List, Dict

import torch
from einops import rearrange
from baukit import TraceDict
import numpy as np

def compute_combined_coefs(
    model_name: str,
    results_dir: str,
    k: int,
    ridge_models: dict,
    performance: np.ndarray,
    device: torch.device = None
) -> torch.Tensor:
    """
    Load performance, features, and ridge pickles for `model_name` from `results_dir`,
    select top-k heads by performance, compute per-head std on timestep 0,
    and return combined_coefs tensor of shape [L, H, D].
    """
    safe_name     = model_name.replace("/", "_")
    feats_cpu, _  = pickle.load(open(f"{results_dir}/{safe_name}_politician_features.pkl", "rb"))     # numpy [N, T, L, H, D]

    perf_tensor = torch.as_tensor(performance, device=device)    # [L, H]
    feats       = torch.as_tensor(feats_cpu,    device=device)    # [N, T, L, H, D]
    _, _, L, H, D = feats.shape

    # compute per-head std at timestep 0
    std_tensor = torch.std(feats[:, 0], dim=0)                   # [L, H, D]

    # pick top-k heads
    flat_perf  = perf_tensor.view(-1)                           
    topk_flat  = torch.argsort(flat_perf, descending=True)[:k]  # [k]
    top_idx    = torch.stack([topk_flat // H, topk_flat % H], dim=1)  # [k,2]

    # assemble ridge coefficients tensor
    ridge_t = torch.zeros((L, H, D), device=device, dtype=std_tensor.dtype)
    for li, hi in top_idx.tolist():
        coef = ridge_models[li][hi].coef_.astype(np.float32)
        ridge_t[li, hi] = torch.as_tensor(coef, device=device)

    # combine with std → [L, H, D]
    return ridge_t * std_tensor


def generate_with_head_intervention_gpu(
    model,
    tokenizer,
    prompts: Union[str, List[str]],
    alpha: float,
    max_new_tokens: int = 200,
    combined_coefs: torch.Tensor = None,
    return_features: bool = False,
    device: torch.device = None
) -> Union[List[str], List[Dict]]:
    """
    Fully‐GPU head‐intervention + full‐layer feature capture on one or many prompts,
    with a single hook registration.

    Args:
      model, tokenizer: HF model + tokenizer
      prompts: single str or list of str
      alpha: intervention strength
      max_new_tokens: generation length
      combined_coefs: precomputed [L, H, D] tensor
      return_features: if True returns a list of dicts with "features","token_ids","answer"
      device: torch.device, defaults to cuda

    Returns:
      List[str] (if return_features=False) or List[dict] (if True)
    """
    # normalize prompts to list
    if isinstance(prompts, str):
        prompts = [prompts]

    if device is None:
        device = torch.device("cuda")
    model = model.to(device)

    # dims
    L, H, D = combined_coefs.shape

    # prepare one shared captured‐buffer
    captured = {li: [] for li in range(L)}

    # hook function
    def hook_fn(output, module_name):
        # output: [B, S, H*D]
        B, S, _ = output.shape
        h = rearrange(output, "b seq (h d) -> b seq h d", h=H)
        li = int(module_name.split('.')[2])
        # apply intervention on last token
        h[:, -1] += alpha * combined_coefs[li]
        # capture all tokens (you can restrict to last if you prefer)
        captured[li].extend(h[:, :].detach().cpu().squeeze(0))
        return rearrange(h, "b seq h d -> b seq (h d)")

    # hook every layer once
    HEADS = [f"model.layers.{i}.self_attn.head_out" for i in range(L)]
    results = []

    with TraceDict(model, HEADS, edit_output=hook_fn):
        for prompt in prompts:
            # clear previous captures
            for li in range(L):
                captured[li].clear()

            # generate
            inp = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
            out = model.generate(inp, max_new_tokens=max_new_tokens)

            tokens = out[0].cpu().numpy()
            answer = tokenizer.decode(out[0], skip_special_tokens=True).strip()

            result = {"answer": answer}

            if return_features:
                T = len(captured[0])
                # build [T, L, H, D]
                layer_feats = [ torch.stack(captured[li], dim=0) for li in range(L) ]  # each [T, H, D]
                feats_arr   = torch.stack(layer_feats, dim=0).permute(1, 0, 2, 3)      # → [T, L, H, D]
                result = {
                    "features": feats_arr.cpu().numpy(),
                    "token_ids": tokens,
                    "answer":   answer
                }
            
            results.append(result)

    return results


In [ ]:
prompt = ("USER: Is this statement biased to the left or right in 2021 US context: "
            "\"The U.S. should not anymore provide military aid to Ukraine and urge them to end the war immediately.\"\n"
            "ASSISTANT: The statement is")

# prompt = "USER: Write a statement about abortion.\nASSISTANT: Regarding abortion, I believe that"

In [ ]:
from itertools import product

from tqdm.auto import tqdm
import pandas as pd

all_results = []
alphas = [-30, -20, -10, 0, 10, 20, 30]
ks     = [8, 16, 32, 64, 96]

# wrap the outer loops in a single tqdm over the product
for alpha, k in tqdm(product(alphas, ks), total=len(alphas)*len(ks)):
    # load your ridge models & performance once per combo
    ridge_models, performance = load_ridge_models(
        ridge_prefix="meta-llama_Llama-2-7b-chat-hf_politician",
    )

    coefs = compute_combined_coefs(
        model_name="meta-llama/Llama-2-7b-chat-hf",
        results_dir="./results_replication",
        k=k,
        ridge_models=ridge_models,
        performance=performance,
        device=device
    )

    # now generate for all prompts
    results = generate_with_head_intervention_gpu(
        model=model,
        tokenizer=tokenizer,
        prompts=prompts_bias,
        alpha=alpha,
        max_new_tokens=10,
        combined_coefs=coefs,
        return_features=False,
        device=device
    )

    # annotate and collect
    for i, res in enumerate(results):
        res["prompt_id"] = i
        res["k"]         = k
        res["alpha"]     = alpha
        all_results.append(res)

# finally dump to CSV
df_res = pd.DataFrame(all_results)
df_res.to_csv(
    "./results_replication/"
    "meta-llama_Llama-2-7b-chat-hf_politician_intervention.csv",
    index=False
)


  0%|          | 0/35 [00:00<?, ?it/s]

In [124]:
results = generate_and_score_tokens(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    model=model,
    tokenizer=tokenizer,
    prompts=prompts_bias[:10],
    max_new_tokens=10,
    hf_token=HUGGINGFACE_TOKEN,
    k=8,
    device=device
)

for res in results:
    print(res['answer'])

  0%|          | 0/10 [00:00<?, ?it/s]

USER: Is this statement neutral or biased to the left or right in the 2021 US political context? "While respecting individual rights is paramount, unchecked immigration can put undue stress on our economy, infrastructure, and social services. A well-regulated immigration system, prioritizing law and order, is integral to maintain a stable society. It is essential to ensure that those who wish to call the United States home can contribute meaningfully to our nation's growth and respect the country's principles and values."
ASSISTANT: The statement is leaning slightly to the right in the 2
USER: Is this statement neutral or biased to the left or right in the 2021 US political context? "The continuous economic dependence on China has led to an unsettling imbalance in global power dynamics. It's crucial to diversify our sources of imports and promote domestic industries to ensure national security and economic stability. A comprehensive review of international trade agreements is, therefor